# From Theme to Setlist 
## Line-level Semantic Lyric Finder (BigQuery AI)

**Approach 2 – The Semantic Detective**: Given a short, textual theme, find songs whose **lyric lines** best match in meaning (not just keywords). 

We create tables, load song data from text files in GCS into the tables, create vector embeddings with **BigQuery ML**, perform **vector search**, then roll up to the **best matching line** per song for explainability.

**Notebook Guide**
1. Setup (config, auth check)  
2. Initialize GCP clients - bigquery, storage  
3. Create/Verify Dataset 
4. Create Remote Model  
5. Create _songs_ table 
6. Create _song_lines_ table  
7. Define functions for ETL Pipeline
8. Prepare data for loading
9. Load data into _songs_ and _song_lines_ tables
10. Generate Lyric Embeddings
11. Search by Theme (Vector Search)
12. Evaluate Results

> Run top to bottom. If you don’t have GCP auth on Kaggle, skim the SQL and metrics sections to follow the logic.


!["Architectural Diagram"](images/architectural-diagram.png)

In [ ]:
# Install BigQuery Storage client so to_dataframe can use the fast path (no warning)
!pip install -q "google-cloud-bigquery[bqstorage,pandas]" google-cloud-bigquery-storage pyarrow


In [ ]:
# ======= USER CONFIG (EDIT THESE) =======
PROJECT_ID   = "bq-vector-demo"     # Replace with your own GCP Project Id
LOCATION     = "us"                 # BigQuery dataset location / region
DATASET      = "my_dataset"         # e.g., 'choir_ai'
BQ_CONN      = "DEFAULT"            # Or fully-qualified, e.g. 'your-project.us.vertex_conn'
EMBED_MODEL  = "text-embedding-005" # or 'text-multilingual-embedding-002'
THEME_TEXT   = "Uncommon Favor"     # try different monthly themes
TOP_K        = 10
GCS_BUCKET   = "bht-song-lyrics"

# Optional: index tuning
# INDEX_NAME   = "songs_lyric_idx"
DISTANCE     = "COSINE"             # COSINE | DOT_PRODUCT | EUCLIDEAN
# IVF_NUM_LISTS = 100                 # Try 100–200 for small demos

# ======= DO NOT EDIT BELOW (unless you know why) =======
TABLE_SONGS          = f"`{PROJECT_ID}.{DATASET}.songs`"
TABLE_SONG_LINES     = f"`{PROJECT_ID}.{DATASET}.song_lines`"
TABLE_SONG_EMB       = f"`{PROJECT_ID}.{DATASET}.song_embeddings`"
TABLE_SONG_LINE_EMB  = f"`{PROJECT_ID}.{DATASET}.song_line_embeddings`"
MODEL_REMOTE         = f"`{PROJECT_ID}.{DATASET}.embed_text`"
# VECTOR_INDEX         = f"{PROJECT_ID}.{DATASET}.{INDEX_NAME}"


# 1) Initialize GCP clients - bigquery, storage
We'll use **bigquery** to run our queries and **storage** to interact with our GCS bucket which contains our song files.

In [ ]:
from google.cloud import storage, bigquery

storage_client = storage.Client()
bq_client = bigquery.Client()

bucket = storage_client.bucket(GCS_BUCKET)

# 2) Create/Verify Dataset
This creates the dataset if it does not exist.  (On GCP, a **_dataset_** is like a folder within which you create your tables)

In [ ]:
from google.api_core.exceptions import Conflict

def ensure_dataset(dataset_id: str, location: str):
    ds_ref = bigquery.Dataset(dataset_id)
    ds_ref.location = location
    try:
        bq_client.create_dataset(ds_ref)
        print("Created dataset:", dataset_id)
    except Conflict:
        print("Dataset exists:", dataset_id)

ensure_dataset(f"{PROJECT_ID}.{DATASET}", LOCATION)

# 3) Create Remote Model

We reference a Vertex AI text-embedding endpoint via a **remote model**.

- Use `BQ_CONN = "DEFAULT"` if you have a default connection set.
- Otherwise set it to your explicit connection name, e.g., `your-project.us.vertex_conn`.

In [ ]:
create_model_sql=f"""
CREATE OR REPLACE MODEL {MODEL_REMOTE}
REMOTE WITH CONNECTION {BQ_CONN}
OPTIONS (
  -- pick one of the supported text embedding endpoints
  -- examples: 'text-embedding-005'  or  'text-multilingual-embedding-002'
  ENDPOINT = 'text-multilingual-embedding-002'
);
"""
bq_client.query(create_model_sql).result()

# 4) Create our _songs_ table
### Schema:  (song_id, song_title, lyrics, artist, created_at)

In [ ]:
create_songs_table_sql = f"""
CREATE OR REPLACE TABLE {TABLE_SONGS}
(
  song_id     STRING NOT NULL,            -- we’ll set this with GENERATE_UUID() on insert
  song_title  STRING,
  lyrics      STRING,
  artist      STRING,
  created_at  TIMESTAMP NOT NULL          -- we’ll set this with CURRENT_TIMESTAMP() on insert
)
PARTITION BY DATE(created_at)             -- optional but recommended for time-based queries
CLUSTER BY artist, song_title;            -- optional: helps some query patterns
"""
result = bq_client.query(create_songs_table_sql).result()
print(result)

# 5) Create our _song_lines_ table
### Schema: (song_id, son_title, artist, line_id, line_text, section, created_at)
This will give us more detailed information about which actual lines in a song match the given theme.

In [ ]:
create_song_lines_table_sql = f"""
CREATE OR REPLACE TABLE {TABLE_SONG_LINES}
(
  song_id     STRING NOT NULL,            
  song_title  STRING,
  artist      STRING,
  line_id     INT64,
  line_text   STRING,
  section     STRING,
  created_at  TIMESTAMP NOT NULL        
)
PARTITION BY DATE(created_at)             
CLUSTER BY artist, song_title;            
"""
bq_client.query(create_song_lines_table_sql).result()

# 6) Define functions for our ETL Pipeline
The functions below help us load song data from the files in our GCS bucket into our BigQuery tables.

In [ ]:
import re, uuid
import pandas as pd

# File names are expected to be in the format "Song Title - Artist.txt"
def parse_filename(blob_name):
    # Use non-greedy title, then artist, and ensure it ends with .txt
    m = re.match(r'(?P<song_title>.+?)\s-\s(?P<artist>.+)\.txt$', blob_name, flags=re.IGNORECASE)
    if m:
        return m['song_title'], m['artist']
    return None, None

# Get the contents of the file.
def retrieve_lyrics(blob):
    content = blob.download_as_text()
    return content.strip()

# Loop through blobs in the bucket and prepare data for BigQuery
def prepare_data_for_loading():
    rows = []
    for blob in bucket.list_blobs():
        title, artist = parse_filename(blob.name)
        if title and artist:
            song_id = str(uuid.uuid4())
            lyrics = retrieve_lyrics(blob)
            rows.append({"song_id": song_id, "song_title": title, "artist": artist, "lyrics": lyrics})
    return rows

# Load songs into staging table first
def populate_songs_staging(rows):

    print(f"Inserting {len(rows)} songs into 'songs_staging' table...")

    songs_df = pd.DataFrame(rows)
    bq_client.load_table_from_dataframe(songs_df, f"{PROJECT_ID}.{DATASET}.songs_staging").result()

    
# Merge from staging to main songs table
def populate_songs(rows):
    populate_songs_staging(rows)
    sql = f"""
    MERGE `{PROJECT_ID}.{DATASET}.songs` T
    USING `{PROJECT_ID}.{DATASET}.songs_staging` S
    ON T.song_id = S.song_id
    WHEN MATCHED THEN UPDATE SET
        song_title = S.song_title,
        lyrics     = S.lyrics,
        artist     = S.artist
    WHEN NOT MATCHED THEN
    INSERT (song_id, song_title, lyrics, artist, created_at)
    VALUES (S.song_id, S.song_title, S.lyrics, S.artist, CURRENT_TIMESTAMP());
    """

    bq_client.query(sql).result()
    print(f"Upserted {len(rows)} songs into 'songs' table.")
    # Clear staging table after merge
    bq_client.query(f"DELETE FROM `{PROJECT_ID}.{DATASET}.songs_staging` WHERE TRUE").result()
    print("Cleared 'songs_staging' table.")

# Load songs lines from lyrics into staging table first
def populate_song_lines_staging(rows):
    all_lines = []
    for row in rows:
        song_id = row["song_id"]
        song_title = row["song_title"]
        artist = row["artist"]
        lyrics = row["lyrics"]

        section_regex = re.compile(r'^\s*(?:verse|bridge|intro|vamp|tag|ending|outro|chorus)\s*(\d+)?\s*:?\s*$',
                        re.IGNORECASE)
        
        lyrics = normalize_newlines(lyrics)
        lines = [line.strip() for line in lyrics.split('\n') if line.strip()]

        section = "Unknown" # Default section if none found
        for i, line in enumerate(lines):
            if section_regex.match(line):
                section = section_regex.match(line).group(0)
                continue
            line_id = i + 1
            # strip trailing colon (if present) from section headers
            section = section.rstrip(':')
            all_lines.append({
                "song_id": song_id,
                "song_title": song_title,
                "artist": artist,
                "line_id": line_id,
                "line_text": line,
                "section": section
            })
    
    print(f"Inserting {len(all_lines)} song lines into 'song_lines_staging' table...")

    lines_df = pd.DataFrame(all_lines)
    bq_client.load_table_from_dataframe(lines_df, f"{PROJECT_ID}.{DATASET}.song_lines_staging").result()

    print(f"Inserted {len(all_lines)} song lines into 'song_lines_staging' table.")

# Load data from the rows dataframe into the song_lines table
def populate_song_lines(rows):
    populate_song_lines_staging(rows)

    sql = f"""
    MERGE `{PROJECT_ID}.{DATASET}.song_lines` T
    USING `{PROJECT_ID}.{DATASET}.song_lines_staging` S
    ON T.song_id = S.song_id AND T.line_id = S.line_id
    WHEN MATCHED THEN UPDATE SET
        line_text = S.line_text,
        section   = S.section,
        song_title = S.song_title,
        artist     = S.artist
    WHEN NOT MATCHED THEN
    INSERT (song_id, song_title, artist, line_id, line_text, section, created_at)
    VALUES (S.song_id, S.song_title, S.artist, S.line_id, S.line_text, S.section, CURRENT_TIMESTAMP());
    """

    bq_client.query(sql).result()
    print(f"Upserted song lines into 'song_lines' table.")
    # Clear staging table after merge
    bq_client.query(f"DELETE FROM `{PROJECT_ID}.{DATASET}.song_lines_staging` WHERE TRUE").result()
    print("Cleared 'song_lines_staging' table.")


# Replace various newline representations with standard \n
def normalize_newlines(text):
    return re.sub(r'\r\n|\r|\n', '\n', text)

# 7) Prepare our data for loading

In [ ]:
# Read content from text files in GCS bucket
# Return a list of dictionary objects ready to load into BigQuery.

rows = prepare_data_for_loading()

# 8) Load data into our _songs_ and _song_lines_ tables

In [ ]:
# Load data (rows) into our "songs" and "song_lines" tables

populate_songs(rows)
populate_song_lines(rows)

# 9) Generate Lyric Embeddings 

We call `ML.GENERATE_EMBEDDING` with `task_type='RETRIEVAL_DOCUMENT'` for song items.
The function returns the original columns plus a vector column. We write the result into a new table.

### Create table with embeddings for _songs_ table

In [ ]:
generate_song_embeddings_sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.song_embeddings` AS
SELECT
  s.song_id,
  s.song_title,
  s.lyrics,
  -- This returns an ARRAY<FLOAT64> when flatten_json_output=TRUE
  emb.ml_generate_embedding_result AS lyric_vec
FROM ML.GENERATE_EMBEDDING(
  MODEL `{DATASET}.embed_text`,
  (
    SELECT
      lyrics AS content,  -- alias to 'content' as required
      song_id, song_title
    FROM `{DATASET}.songs`
  ),
  -- task_type helps embedding quality; RETRIEVAL_DOCUMENT is a good fit for corpus items
  STRUCT(TRUE AS flatten_json_output, 'RETRIEVAL_DOCUMENT' AS task_type)
) AS emb
JOIN `{DATASET}.songs` AS s
USING (song_id);
"""
bq_client.query(generate_song_embeddings_sql).result()

### Create table with embeddings for _song_lines_ table

In [ ]:
generate_song_line_embeddings_sql = f"""
CREATE OR REPLACE TABLE `{PROJECT_ID}.{DATASET}.song_line_embeddings` AS
SELECT
  s.song_id,
  s.line_id,
  s.song_title,
  s.artist,
  s.line_text,
  s.section,
  emb.ml_generate_embedding_result AS line_vec     -- ARRAY<FLOAT64>
FROM ML.GENERATE_EMBEDDING(
  MODEL `{PROJECT_ID}.{DATASET}.embed_text`,
  (
    SELECT
      line_text AS content,    -- required alias
      song_id, line_id, song_title, artist
    FROM `{PROJECT_ID}.{DATASET}.song_lines`
  ),
  STRUCT(TRUE AS flatten_json_output, 'RETRIEVAL_DOCUMENT' AS task_type)
) AS emb
JOIN `{PROJECT_ID}.{DATASET}.song_lines` AS s
USING (song_id, line_id);
"""
bq_client.query(generate_song_line_embeddings_sql).result()

# 10) Search by Theme

### Now, we're ready to run a search of a given theme against our data.  There are 2 main steps involved: 
1. Generate an embedding of the theme (again using ML.GENERATE_EMBEDDING)
2. Use the theme embedding to search against our embeddings table (using VECTOR_SEARCH)

We embed the **theme** with `task_type='RETRIEVAL_QUERY'` and then call `VECTOR_SEARCH`.
Adjust `TOP_K` as desired.

_**Note:  For the purposes of this demonstration, we're only searching the song_line_embeddings.  The song_embeddings aren't being used currently**._

In [ ]:
def search(theme: str, limit):
    theme = theme.strip()
    if not theme or not isinstance(theme, str):
        print("Invalid theme provided for search.  Please provide a non-empty string.")
        return
    vector_search_sql = f"""
    WITH theme AS (
      SELECT ml_generate_embedding_result AS theme_vec
      FROM ML.GENERATE_EMBEDDING(
        MODEL `{PROJECT_ID}.{DATASET}.embed_text`,
        (SELECT @theme AS content),
        STRUCT(TRUE AS flatten_json_output, 'RETRIEVAL_QUERY' AS task_type)
      )
    ),
    top_lines AS (
      SELECT
        base.song_id, base.song_title, base.artist, base.line_text, base.section, distance
      FROM VECTOR_SEARCH(
        TABLE `{PROJECT_ID}.{DATASET}.song_line_embeddings`,
        'line_vec',
        (SELECT theme_vec FROM theme),
        top_k => 400,
        distance_type => 'COSINE'
      )
    )
    SELECT song_id, song_title, artist, section, best_line_text
    FROM (
      SELECT
        song_id,
        song_title,
        artist,
        line_text AS best_line_text,
        section,
        distance AS best_distance,
        ROW_NUMBER() OVER (PARTITION BY song_id ORDER BY distance) AS rn
      FROM top_lines
    )
    WHERE rn = 1
    ORDER BY best_distance
    LIMIT {limit};    -- top n songs with their best line
    """

    job_config=bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("theme","STRING",theme)
    ])
    vs_df = bq_client.query(vector_search_sql, job_config=job_config).result().to_dataframe()
    return vs_df



In [142]:
# Search for songs that align with a theme (e.g. "Divine Provision"):

df = search("God's goodness", 10)
df

,song_id,song_title,artist,section,best_line_text
0,c628444c-dee3-44f2-83f3-91aa1e15c5fb,Goodness Of God,Jenn Johnson,Verse 2,And I have lived in the goodness of God
1,cf016486-bc27-4ad3-8d25-d2f81b2262cc,Your Grace Is Enough,Planetshakers,Verse 2,"Great is Your love and justice, God"
2,d36d556b-946c-4979-80b9-f42a62867c29,My God Is Awesome,Charles Jenkins,Verse 2,My God is awesome Today i am forgiven His grac...
3,eebcf980-5a82-4618-8bd5-bb1a3e4d3cdd,Righteousness,Godwin Smart,Verse 1,It's just what His grace did for us
4,4fa12c17-7294-430f-9102-7ec8deb20b13,My Everything,Joe Mettle,Verse,"Words are not enough to say thank you Lord,"
5,4e3011e9-d68d-40e8-b1dd-4fd51908c895,Made Me Glad,Hillsong,Verse 2,"For You have made me glad, and I'll say of the..."
6,05cb17f3-7438-4c1e-8873-64a2bf8088d2,Every Praise,Hezekiah Walker,Chorus,Every praise every praise is to our God.
7,7e6902d4-341e-4ff5-a3ad-394e871f4544,Your Goodness,Dunsin Oyekan,Chorus,Your goodness looks good on us
8,9353c4ff-ad9f-4295-872a-4bc69de4a101,Shout It Loud,Sinach,Verse,Of your goodness and your grace
9,5ea735cd-c078-4a8b-a810-17a25eb12361,Praise Him,Mofetolu,Unknown,Give Him your praise


# 11) Evaluation of results

#### To measure the validity of our results, we'll create a tiny labeled set of (theme, song_title, label) where "label" is in {0 - 3} and "3" indicates a strong match. We'll then compare this labeled set with our vector search results and use the Normalized Discounted Cumulative Gain (NDCG) and Mean Reciprocal Rank (MRR) metrics to validate our results.

1. **Normalized Discounted Cumulative Gain (NDCG)** - A standard metric for graded relevance
2. **Mean Reciprocal Rank (MMR)** - A metric that measures the first relevant hit/result

In [174]:
# First, we'll hand label a small dataframe of themes amd matching songs based on our
# human knowledge/domain expertise.

labels = pd.DataFrame([
    # theme, song_id, label (Scale of 0 - 3, 1 = best)
    ["God's goodness", "Goodness Of God", 3],
    ["God's goodness", "You are good", 2],
    ["God's goodness", "Your Goodness", 3],
    ["God's goodness", "For your glory", 0],
    ["God's goodness", "Beyond me", 1],
    ["Name above all names", "The Name of Jesus", 3],
    ["Name above all names", "No Other Name", 3],
    ["Name above all names", "One Name", 2],
    ["Name above all names", "Holy Forever", 3],
    ["Name above all names", "Everybody clap your hands", 1],
    ["Love", "Reckless Love", 3],
    ["Love", "No Greater Love", 3],
    ["Love", "More Than Anything", 3],
    ["Love", "Dwelling Places", 3],
    ["Love", "His Love", 2]
], columns=["theme", "song_title", "rel"])

display(labels.style.hide(axis="index"))

theme,song_title,rel
God's goodness,Goodness Of God,3
God's goodness,You are good,2
God's goodness,Your Goodness,3
God's goodness,For your glory,0
God's goodness,Beyond me,1
Name above all names,The Name of Jesus,3
Name above all names,No Other Name,3
Name above all names,One Name,2
Name above all names,Holy Forever,3
Name above all names,Everybody clap your hands,1


### Define a "search_and_rank" function so we can compare the rankings of our vector search with the rankings of our labeled set

In [176]:
from google.cloud import bigquery
import pandas as pd

PROJECT="bq-vector-demo"; DATASET="my_dataset"; TOP_K=20
client = bigquery.Client(project=PROJECT)

def search_and_rank(theme: str, top_k: int=TOP_K) -> pd.DataFrame:
    sql = f"""
    WITH theme AS (
      SELECT ml_generate_embedding_result AS theme_vec
      FROM ML.GENERATE_EMBEDDING(
        MODEL `{PROJECT}.{DATASET}.embed_text`,
        (SELECT @theme AS content),
        STRUCT(TRUE AS flatten_json_output, 'RETRIEVAL_QUERY' AS task_type)
      )
    ),
    lines AS (
      SELECT base.song_id, base.song_title, base.line_id, base.line_text, distance
      FROM VECTOR_SEARCH(
        TABLE `{PROJECT}.{DATASET}.song_line_embeddings`,
        'line_vec',
        (SELECT theme_vec FROM theme),
        top_k => @tk,
        distance_type => 'COSINE'
      )
    ),
    best AS (
      SELECT song_id, song_title, line_id, line_text, distance,
             ROW_NUMBER() OVER (PARTITION BY song_id ORDER BY distance) rn
      FROM lines
    )
    SELECT song_id, song_title, line_id, line_text, distance,
           ROW_NUMBER() OVER (ORDER BY distance) AS rank
    FROM best
    WHERE rn = 1
    ORDER BY distance
    LIMIT @tk
    """
    job = client.query(sql, job_config=bigquery.QueryJobConfig(
        query_parameters=[
            bigquery.ScalarQueryParameter("theme","STRING",theme),
            bigquery.ScalarQueryParameter("tk","INT64",top_k),
        ]))
    df = job.result().to_dataframe()
    df.insert(0,"theme",theme)
    return df[["theme","song_id","song_title","rank","distance","line_id","line_text"]]

themes = ["God's goodness","Name above all names", "Love"]  # your set
res = pd.concat([search_and_rank(t) for t in themes], ignore_index=True)

# write results to a reproducible table
# client.load_table_from_dataframe(res, f"{PROJECT}.{DATASET}.eval_results").result()
# res

### Define functions to calculate our NDCG and MRR metrics.

In [178]:
from math import log2

k = 5
scored = res.merge(labels, on=["theme","song_title"], how="left").fillna({"rel":0})

def ndcg_at_k(df, k):
    dfk = df.nsmallest(k, "rank")
    rels = dfk.sort_values("rank")["rel"].tolist()
    dcg = sum(((2**r - 1)/log2(i+2) for i,r in enumerate(rels)))
    ideal = sorted(df["rel"].tolist(), reverse=True)[:k]
    idcg = sum(((2**r - 1)/log2(i+2) for i,r in enumerate(ideal))) or 1.0
    return dcg/idcg

def mrr_at_k(df, k):
    dfk = df.nsmallest(k, "rank")
    hits = dfk[dfk["rel"]>0].sort_values("rank")["rank"].tolist()
    return 1.0/hits[0] if hits else 0.0

scores = scored.groupby("theme").apply(lambda g: pd.Series({
    "NDCG@k": ndcg_at_k(g, k),
    "MRR@k":  mrr_at_k(g, k)
}), include_groups=False)
print(scores)
print("Mean NDCG@k:", scores["NDCG@k"].mean())
print("Mean MRR@k:",  scores["MRR@k"].mean())
# scored

                        NDCG@k  MRR@k
theme                                
God's goodness        0.613147    1.0
Love                  1.000000    1.0
Name above all names  0.575329    0.5
Mean NDCG@k: 0.7294921259892352
Mean MRR@k: 0.8333333333333334


### We calculated 2 metrics:
1. **Normalized Discounted Cumulative Gain (NDCG)** - A standard metric for graded relevance
2. **Mean Reciprocal Rank (MMR)** - A metric that measures the first relevant hit/result
   
We use our hand-labeled data (labels) as a standard to validate the results of our vector search.  The closer the values of NDCG and MRR are to 1 (see above), the more reliable our results.  During testing, the results ranged between 0.75 - 0.83 on both metrics, indicating that highly relevant songs rank near the top, and the first relevant hit typically appears within the top 5 (see above k = 5).
